<a href="https://colab.research.google.com/github/kavyasalian/ML_ANN/blob/master/neural_network_on_wine_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
wine=load_wine()
print(wine.DESCR)

df=pd.DataFrame(data=np.c_[wine['data'],wine['target']],columns=wine['feature_names'] + ['target'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [51]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dsigmoid(x):
    return x * (1-x)

def relu(x):
     return abs(x) * (x > 0)

def drelu(x):
     return 1. * (x > 0.)

def lrelu(x):
    return np.where(x > 0., x, x * 0.01)

def dlrelu(x):
    dx = np.ones_like(x)
    dx[x < 0.] = 0.01
    return dx


def tanh(x):
    return np.tanh(x)

def dtanh(x):
    return 1.0 - (np.power(np.tanh(x),2))


In [52]:
df=pd.get_dummies(df, columns=['target'])


In [53]:
y=df.loc[:,['target_0.0','target_1.0','target_2.0']]

In [54]:
x=df.drop(['target_0.0','target_1.0','target_2.0'],axis=1)

In [55]:
x=x.values
x = (x-x.min())/ (x.max()- x.min())
y=y.values


In [56]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2)

In [57]:
def feed_forward(data_in,w0,w1,b0,b1):
    layer0 = data_in
    layer1 =sigmoid(np.dot(layer0,w0)+b0)
   
    layer2= sigmoid(np.dot(layer1,w1)+b1)
    
    return layer0,layer1,layer2

In [58]:
def back_propogation(layer0,layer1,layer2,actual_y,w0,w1,b0,b1,learning_rate,i):
    
    l2_error = layer2 - actual_y
    l2_delta = l2_error * dsigmoid(layer2) 
    dh2 = np.dot(layer1.T,l2_delta)
    
    l1_error = np.dot(l2_delta,w1.T)
    l1_delta = l1_error * dsigmoid(layer1) 
    dh1 = np.dot(layer0.T, l1_delta)
    
    w1 = w1 - (learning_rate * dh2)
    w0 = w0 - (learning_rate * dh1)
    
    b1=b1 - (learning_rate * np.mean(l2_delta))
    b0=b0 - (learning_rate * np.mean(l1_delta))
    
    if i%10 ==0 and (i!=0):
        loss=np.mean(np.power(layer2 - actual_y,2))
        loss_curve.append(loss)
        iters.append(int(i))
        
        if i%100==0:
            print('\n',int(i),loss)
    return w0,w1,b0,b1

In [59]:
w0 = np.random.random((13,4))-1
w1 = np.random.random((4,3))-1

b0 = np.random.random((1,4))-1
b1 = np.random.random((1,3))-1

In [60]:
epochs=50000
loss_curve = []
iters = []

In [61]:
for  i in range(epochs):
    layer0,layer1,layer2=feed_forward(xtrain,w0,w1,b0,b1)
    w0,w1,b0,b1= back_propogation(layer0,layer1,layer2,ytrain,w0,w1,b0,b1,0.5,i)


 100 0.17060470879251433

 200 0.15008546664586794

 300 0.140645033883074

 400 0.13862667703340004

 500 0.1370873608392036

 600 0.13517613581122065

 700 0.13300084144947594

 800 0.13081678046165013

 900 0.12887680071134217

 1000 0.12669119297569217

 1100 0.1321338512206649

 1200 0.12173769424671098

 1300 0.12748995589308987

 1400 0.1415053188798176

 1500 0.13561805518732836

 1600 0.11673764902338034

 1700 0.13999068109127216

 1800 0.12429869849070872

 1900 0.1429367243946097

 2000 0.1314155487131565

 2100 0.12053737780834954

 2200 0.12932022237425186

 2300 0.1696488809912208

 2400 0.11761446040296387

 2500 0.11759155853299534

 2600 0.12673599467287508

 2700 0.1090598478074511

 2800 0.1106016004463585

 2900 0.11970384923603834

 3000 0.10701328406255618

 3100 0.13769317101505005

 3200 0.1196805279722694

 3300 0.12180539770452041

 3400 0.1309294033174683

 3500 0.1465445219499583

 3600 0.1268021590266731

 3700 0.12176667022330077

 3800 0.159482611048287

In [62]:
def accuracy(testx, testy):
  correct = 0
  layer0, layer1, layer2 = feed_forward(testx,w0,w1,b0,b1)
  for i in range(len(testx)):
    if np.argmax(layer2[i]) == np.argmax(testy[i]):
      
      correct +=1
  return f"Accuracy: {correct*100/len(testy)}"

In [63]:
accuracy(xtrain, ytrain)

'Accuracy: 95.77464788732394'

In [64]:
accuracy(xtest, ytest)

'Accuracy: 88.88888888888889'